In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
import lxml
import lxml.etree
import lxml
import lxml.etree
import httplib2
from lxml import html 
from pandas import DataFrame

In [2]:
url = 'https://www.drivendata.org/competitions/'
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')

In [3]:
#Titles for Data driven competation
Title=soup.find_all('h3')
#Price money for competation
Price_Money=soup.find_all("span", attrs={'prize'})

In [4]:
def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.drivendata.org/competitions/', headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'panel'}):
        title =d.find('h3', attrs={'class':''})
        Reward_amount = d.find('span', attrs={'class':'prize'})
        competation_url = d.find('a', href=True)
        Deliverable=d.find('p')
        Winners=d.find('h4') 
        Judge=d.find('small')
        Dummy=d.find('div', attrs={'class':'user-body'})
        all1=[]
        
        if title is not None:
            #print(price.text)
            all1.append(title.text)
        else:
            all1.append('Title is NA')
   
        if Reward_amount is not None:
            #print(price.text)
            all1.append(Reward_amount.text)
        else:
            all1.append('0')
        if competation_url is not None:
            all1.append(competation_url['href'])
        else:
            all1.append('-')
        if Deliverable is not None:
            all1.append(Deliverable.text)
        else:
            all1.append('NA')
        if Winners is not None:
            all1.append(Winners.text)
        else:
            all1.append('NA')
        if Judge is not None:
            all1.append(Judge.text)
        else:
            all1.append('NA')
        if Dummy is not None:
            all1.append(Dummy.text)
        else:
            all1.append('NA')
        alls.append(all1)    
    return alls

In [5]:
results = []
for i in range(1,2):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Title','Reward_amount','competation_url','Deliverable','Winners','Judge','Dummy'])
df.to_csv('file_test.csv', index=False, encoding='utf-8')

df = pd.read_csv("file_test.csv")

df['Platform_name'] = 'https://www.drivendata.org'
df.loc[df['Reward_amount'] != '0','Reward_type']= 'Monetary'
df.loc[df['Reward_amount'] == '0','Reward_type']= 'Non_Monetary'
df['competation_url_2'] = df[['Platform_name', 'competation_url']].apply(lambda x: ''.join(x), axis=1)
df['Leader_board']='yes'
df.drop(['competation_url'],axis=1,inplace=True)
df.loc[df['Judge'] != 'COMPETITION ENDED','COMPETITION_STATUS']= 'OPEN'
df.loc[df['Judge'] == 'COMPETITION ENDED','COMPETITION_STATUS']= 'CLOSED'
df['Group']=df['Dummy'].str.contains('Leaders|1st Place Team')
df.drop(['Dummy'],axis=1,inplace=True)
df.drop(df.head(1).index, inplace=True)

In [6]:
url_2= 'https://www.drivendata.org/competitions/72/predict-wind-speeds/'
response_2 = requests.get(url_2)
content = response_2.content
soup = BeautifulSoup(content)
tree = html.fromstring(content)

In [7]:
Problem_description = []
for j in range(1,40): 
    url=df.competation_url_2[j]
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get(url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    tree = html.fromstring(content)
    for i in soup.find_all('div', attrs={'class':'col-md-8 col-md-pull-4'}):
        if i is None:
            Problem_description.append("NA")
        else:
            Problem_description.append(i.text)
df_2 = pd.DataFrame({'Description':Problem_description})

In [36]:
Winner_a = []
Host = []
for j in range(1,40): 
    url=df.competation_url_2[j]
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get(url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    tree = html.fromstring(content)
    Winner_1 = []
    f = soup.find('tr', attrs={'class':'success'})
    s = soup.find('tr', attrs={'class':'info'})
    t = soup.find('tr', attrs={'class':'warning'})
    J = soup.find('small')
    if f is not None:
        Winner_1.append(f.text)
    else:
        Winner_1.append(0)
    if s is not None:
        Winner_1.append(s.text)
    else:
        Winner_1.append(0)
    if t is not None:
        Winner_1.append(t.text)
    else:
        Winner_1.append(0)
    if J is not None:
        Winner_1.append(J.text)
    else:
        Winner_1.append('NA')
    
    Winner_a.append(Winner_1)
    
    
df_3= DataFrame(Winner_a,columns=['First_Winner','Second_Winner','Third_Winner','Judge'])   

In [37]:
df_4=pd.concat([df_2,df_3], axis=1, ignore_index=False)
df_4.index = np.arange(1, len(df)+1)
df_f=pd.concat([df,df_4], axis=1, ignore_index=False)
df_f['Deliverable']=df_f['Deliverable'].str.replace('\n'," ")
df_f['Description']=df_f['Description'].str.replace('\n'," ")
df_f['First_Winner']=df_f['First_Winner'].str.replace('\n'," ")
df_f['Second_Winner']=df_f['Second_Winner'].str.replace('\n'," ")
df_f['Third_Winner']=df_f['Third_Winner'].str.replace('\n'," ")
df_f

,Title,Reward_amount,Deliverable,Winners,Judge,Platform_name,Reward_type,competation_url_2,Leader_board,COMPETITION_STATUS,Group,Description,First_Winner,Second_Winner,Third_Winner,Judge
1,Wind-dependent Variables: Predict Wind Speeds ...,"$13,000","Throughout a tropical cyclone, hu...",vecxoz,FOR-PRIZE COMPETITION,https://www.drivendata.org,Monetary,https://www.drivendata.org/competitions/72/pre...,yes,OPEN,False,"Overview Tropical cyclones, which include t...",1st $6000 and $8000 Azure credit,2nd $4500 and $6000 Azure credit,3rd $2500 and $4000 Azure credit,Hosted By Radiant Earth Foundation
2,MagNet: Model the Geomagnetic Field,"$30,000",Help NOAA better forecast changes...,mltsw,FOR-PRIZE COMPETITION,https://www.drivendata.org,Monetary,https://www.drivendata.org/competitions/73/noa...,yes,OPEN,False,Overview Help NOAA better forecast changes ...,"1st $15,000","2nd $8,000","3rd $5,000",Hosted By NOAA
3,Differential Privacy Temporal Map Challenge: S...,0,START HERE! Help public safety ag...,rmckenna,FOR-PRIZE COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/74/com...,yes,OPEN,False,Overview Large data sets containing persona...,"1st $15,000","2nd $10,000","3rd $5,000",Hosted By NIST PSCR
4,Differential Privacy Temporal Map Challenge: S...,"$39,000",CALLING PRESCREENED PARTICIPANTS!...,DPSyn_2S,FOR-PRIZE COMPETITION,https://www.drivendata.org,Monetary,https://www.drivendata.org/competitions/75/dei...,yes,OPEN,True,Overview Large data sets containing persona...,"1st $15,000","2nd $10,000","3rd $5,000",Hosted By NIST PSCR
5,Hateful Memes: Phase 1,0,Detecting hateful content present...,distracted boyfriend,FOR-PRIZE COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/64/hat...,yes,OPEN,True,Overview Hateful Memes: Phase 2 is live! In...,"1st $50,000","2nd $25,000","3rd $10,000",Hosted By Facebook
6,United Nations Millennium Development Goals,0,The UN's Millennium Development G...,Ganesh221B,FRIENDLY COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/1/unit...,yes,OPEN,False,"Background In the year 2000, the member stat...",NaN,NaN,NaN,Hosted By DrivenData
7,DengAI: Predicting Disease Spread,0,Using environmental data collecte...,cseuom_SolveSoft,FRIENDLY COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/44/den...,yes,OPEN,True,Challenge Summary Can you predict local epi...,NaN,NaN,NaN,Hosted By DrivenData
8,Pump it Up: Data Mining the Water Table,0,Can you predict which water pumps...,serser,FRIENDLY COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/7/pump...,yes,OPEN,False,Challenge Summary Can you predict which wat...,NaN,NaN,NaN,Hosted By DrivenData
9,Richter's Predictor: Modeling Earthquake Damage,0,Can you predict the level of dama...,inoddy,FRIENDLY COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/57/nep...,yes,OPEN,False,Overview Based on aspects of building locatio...,NaN,NaN,NaN,Hosted By DrivenData
10,Flu Shot Learning: Predict H1N1 and Seasonal F...,0,Can you predict whether people go...,inoddy,FRIENDLY COMPETITION,https://www.drivendata.org,Non_Monetary,https://www.drivendata.org/competitions/66/flu...,yes,OPEN,False,Overview Can you predict whether people got ...,NaN,NaN,NaN,Hosted By DrivenData


In [10]:
#//*[@id="content-page"]/h2[1]
abc = []
for i in soup.find_all('div', attrs={'class':'col-md-8 col-md-pull-4'}):
    if i is None:
        abc.append("NA")
    else:
        abc.append(i.text)

In [11]:
#df_f.to_excel('/Users/sampathgubbala/Downloads.xlsx')
df.to_csv(r'/Users/sampathgubbala/Downloads\data_driven.csv')

In [12]:
#df.loc[df['Reward_amount'] != 'COMPETITION ENDED','COMPETITION_STATUS']= 'OPEN'
#df['Reward_Currency']=df['Reward_amount'].str.contains('$')
#df['Reward_Currency'] = pd.np.where(df.Reward_amount.str.contains("$"), "$",
                                  #  pd.np.where(df.Reward_amount.str.contains("€"), "€",'-'))

In [13]:
Problem_description = []
for i in soup.find_all('div', attrs={'class':'col-md-8 col-md-pull-4'}):
        if i is None:
            Problem_description.append("NA")
        else:
            Problem_description.append(i.text)

In [20]:
Deliverable = []
for i in tree.xpath('//*[@id="leader-title"]/small'):
    if i is None:
        Deliverable.append("NA")
    else:
        Deliverable.append(i.text)

In [31]:
Deliverable

[' Hosted By DrivenData ']

In [25]:
abc = tree.xpath('//*[@id="leader-title"]/small')

In [15]:
soup.find_all('div', attrs={'class':'prize-content'})

[]

In [16]:
list(df)

['Title',
 'Reward_amount',
 'Deliverable',
 'Winners',
 'Judge',
 'Platform_name',
 'Reward_type',
 'competation_url_2',
 'Leader_board',
 'COMPETITION_STATUS',
 'Group']